In [2]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('/home/ralf/models/S-BioELECTRA')
embeddings = model.encode(sentences)
print(embeddings)

[[-0.02665126  0.0990985  -0.03884164 ...  0.03455577  0.01290239
   0.07733775]
 [ 0.10659064  0.20882843  0.07350808 ... -0.01763076 -0.00997995
   0.24828523]]


In [12]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math
import json
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

In [24]:
# Load the dataset
import tensorflow_datasets as tfds
import tensorflow as tf
import pandas as pd

text_key = "originalTitle"
dataset_path = "/home/ralf/IdeaProjects/LitBall-training/EXP-Title+TLDR/"
bsize = 30

stop_words = set(stopwords.words('english'))
def preprocess_sentence(text):
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()

    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]

    return ' '.join(tokens)

farr1 = []
larr1 = []
with open(dataset_path + "ROTVRSV") as file:
    lines = file.readlines()
    for line in lines:
        d = json.loads(line)
        farr1.append(np.asarray(model.encode(preprocess_sentence(d[text_key]))).astype('float32'))
        larr1.append(int(d["label"]))
train_ds = tf.data.Dataset.from_tensor_slices((np.array(farr1), np.array(larr1))).batch(bsize)
print("Finished embedding ROTVRSV")

farr2 = []
larr2 = []
with open(dataset_path + "DENV") as file:
    test_lines = file.readlines()
    for line in test_lines:
        d = json.loads(line)
        farr2.append(np.asarray(model.encode(preprocess_sentence(d[text_key]))).astype('float32'))
        larr2.append(int(d["label"]))
test_ds = tf.data.Dataset.from_tensor_slices((np.array(farr2), np.array(larr2))).batch(bsize)
print("Finished embedding DENV")
print("Made datasets")

Finished embedding ROTVRSV
Finished embedding DENV
Made datasets


In [25]:
def prepare_dataset4(s):
    m = json.loads(s)
    return m

test_cases = list(map(prepare_dataset4, test_lines))
print(len(test_lines))
print("Data loaded.")

# Specify the model.
model_1 = tfdf.keras.RandomForestModel(num_trees=1000, verbose=2, num_threads=3, check_dataset=False)

# Train the model.
model_1.fit(x=train_ds, batch_size=None)
model_1.compile(metrics=["accuracy"])
evaluation = model_1.evaluate(test_ds, batch_size=None)

print(f"BinaryCrossentropyloss: {evaluation[0]}")
print(f"Accuracy: {evaluation[1]}")

8679
Data loaded.
Use /tmp/tmpqx6vxa1k as temporary training directory


Reading training dataset...
Training tensor examples:
Features: Tensor("data:0", shape=(None, 768), dtype=float32)
Label: Tensor("data_1:0", shape=(None,), dtype=int64)
Weights: None


2023-07-29 19:58:03.608088: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [22996]
	 [[{{node Placeholder/_1}}]]


Normalized tensor features:
 {'data:0.0': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'strided_slice:0' shape=(None,) dtype=float32>), 'data:0.1': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'strided_slice_1:0' shape=(None,) dtype=float32>), 'data:0.2': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'strided_slice_2:0' shape=(None,) dtype=float32>), 'data:0.3': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'strided_slice_3:0' shape=(None,) dtype=float32>), 'data:0.4': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'strided_slice_4:0' shape=(None,) dtype=float32>), 'data:0.5': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'strided_slice_5:0' shape=(None,) dtype=float32>), 'data:0.6': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'strided_slice_6:0' shape=(None,) dtype=float32>), 'data:0.7': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, t

Training dataset read in 0:00:05.957837. Found 22996 examples.
Training model...


...

Model trained in 0:02:56.954935
Compiling model...


2023-07-29 20:01:06.708733: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [22996]
	 [[{{node Placeholder/_1}}]]


Model compiled.


2023-07-29 20:01:11.487997: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [8679]
	 [[{{node Placeholder/_1}}]]


290/290 [==============================] - 4s 6ms/step - loss: 0.0000e+00 - accuracy: 0.9054
BinaryCrossentropyloss: 0.0
Accuracy: 0.9054038524627686


In [26]:
p = model_1.predict(test_ds)

#Fish for best cut
for cut in np.arange(0.25, .8, .05):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    N = len(p)
    for i in range(N):
        tru = test_cases[i]["label"] == '1'
        prd = p[i] > cut
        if tru and prd == tru:
            tp = tp + 1
        if tru and prd != tru:
            fn = fn + 1
        if tru == False and prd == tru:
            tn = tn + 1
        if tru == False and prd != tru:
            fp = fp + 1
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1_score = 2 * (precision * recall) / (precision + recall)
    print("cut: {}, acc: {}, prec: {}, rec: {}, f1: {}".format(cut, (tp+tn)/N, precision, recall, f1_score))

290/290 [==============================] - 3s 6ms/step
cut: 0.25, acc: 0.7452471482889734, prec: 0.5036248300860897, rec: 0.9910833704859563, f1: 0.6678684091933303
cut: 0.3, acc: 0.8034335752966931, prec: 0.5693619219839835, rec: 0.9826125724476148, f1: 0.7209682695453059
cut: 0.35, acc: 0.8492913930176288, prec: 0.6363371245261009, rec: 0.9728042799821668, f1: 0.7693935119887166
cut: 0.39999999999999997, acc: 0.8778661136075585, prec: 0.692733789507983, rec: 0.9478377173428444, f1: 0.8004518072289155
cut: 0.44999999999999996, acc: 0.897223182394285, prec: 0.7477081041437477, rec: 0.9090503789567543, f1: 0.820523138832998
cut: 0.49999999999999994, acc: 0.9054038483696278, prec: 0.7933168316831684, rec: 0.8573339277753009, f1: 0.8240839940004286
cut: 0.5499999999999999, acc: 0.906095172254868, prec: 0.8352112676056338, rec: 0.7931341952741864, f1: 0.8136290875828951
cut: 0.5999999999999999, acc: 0.899988477935246, prec: 0.8706199460916442, rec: 0.7200178332590281, f1: 0.788189360663738